In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
import warnings 
warnings.filterwarnings('ignore')
from flaml import logger, logging
logger.setLevel(logging.WARNING)
import random

In [250]:
imp  = pd.read_csv("Imputed_scaled.csv")
miss = pd.read_csv("Missingness_0_scaled.csv")
smote = pd.read_csv("Smote_scaled.csv")
TEST = pd.read_csv("unlabeled.csv")

**In order to make sure the colnames are consistent across dfs**

In [277]:
cols = imp.columns
cols = cols.drop(cols[0])
cols

Index(['age', 'bmi', 'height', 'pre_icu_los_days', 'weight', 'albumin_apache',
       'apache_2_diagnosis', 'apache_3j_diagnosis', 'arf_apache',
       'bilirubin_apache',
       ...
       'apache_4a_icu_death_prob', 'aids', 'cirrhosis', 'diabetes_mellitus',
       'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis', 'hospital_death'],
      dtype='object', length=171)

**X and y from the testing unlabelled data**

In [381]:
test_X = TEST[cols]

In [385]:
testing_X = test_X.drop('hospital_death',axis=1)

In [386]:
testing_X

,age,bmi,height,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,arf_apache,bilirubin_apache,...,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,56.0,21.102439,180.3,0.104167,68.60,3.7,301.0,405.01,0.0,NaN,...,0.13,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,19.500595,145.0,0.230556,41.00,NaN,217.0,1502.02,0.0,NaN,...,0.09,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,66.0,32.518596,170.2,0.140972,94.20,NaN,302.0,1208.06,0.0,NaN,...,0.03,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,64.0,20.798520,182.8,0.002083,69.50,1.9,303.0,211.03,0.0,0.8,...,0.19,0.17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,56.0,NaN,NaN,0.004861,66.40,NaN,217.0,1501.01,0.0,NaN,...,0.50,0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39303,35.0,35.038615,170.2,0.411111,101.50,NaN,218.0,1505.01,0.0,NaN,...,0.01,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39304,63.0,29.352338,175.3,0.218056,90.20,NaN,124.0,305.02,0.0,NaN,...,0.08,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39305,71.0,23.010381,170.0,0.003472,66.50,2.0,113.0,502.01,0.0,1.0,...,0.13,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
39306,57.0,32.987948,185.4,0.147917,113.39,NaN,110.0,104.01,0.0,NaN,...,0.04,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [283]:
imp = imp[cols]
miss = miss.drop(miss.columns[0],axis=1)
smote = smote.drop(smote.columns[0],axis=1)

In [285]:
miss.columns

Index(['age', 'bmi', 'height', 'pre_icu_los_days', 'weight', 'albumin_apache',
       'apache_2_diagnosis', 'apache_3j_diagnosis', 'arf_apache',
       'bilirubin_apache',
       ...
       'apache_4a_icu_death_prob', 'aids', 'cirrhosis', 'diabetes_mellitus',
       'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis', 'hospital_death'],
      dtype='object', length=171)

In [142]:
len(imp.columns)

171

**Splits for the imputed dataframe**

In [143]:
imp_X = imp.drop("hospital_death",axis=1)
imp_y = imp['hospital_death']
random.seed(10)
imp_x_train, imp_x_test, imp_y_train, imp_y_test = train_test_split(imp_X,imp_y,test_size=.3, random_state=42)

Once we get predictions, we need to compare w/ race and gender and examine the discrepancies where predicted outcome is different from actual and the only difference is race/gender etc.

In [144]:
svc_poly = svm.SVC(kernel='poly')
svc_rbf = svm.SVC(kernel='rbf')
svc_sig = svm.SVC(kernel='sigmoid')
svc_lin = svm.LinearSVC()

**SVC w/ Linear, poly, rbf, sigmoid**

In [145]:
svc_lin.fit(imp_x_train,imp_y_train)
svc_lin_pred = svc_lin.predict(imp_x_test)
sc_svc_lin = svc_lin.score(imp_x_test,imp_y_test)

In [146]:
svc_poly.fit(imp_x_train,imp_y_train)
svc_pol_pred = svc_poly.predict(imp_x_test)
sc_svc_pol = svc_poly.score(imp_x_test,imp_y_test)

In [147]:
svc_rbf.fit(imp_x_train,imp_y_train)
svc_rbf_pred = svc_rbf.predict(imp_x_test)
sc_svc_rbf = svc_rbf.score(imp_x_test,imp_y_test)

In [148]:
svc_sig.fit(imp_x_train,imp_y_train)
svc_sig_pred = svc_sig.predict(imp_x_test)
sc_svc_sig = svc_sig.score(imp_x_test,imp_y_test)

**Score for imp test**

In [149]:
print("Linear kernel:", sc_svc_lin)
print("Polynomial kernel:", sc_svc_pol)
print("Radio Base Frequency kernel:", sc_svc_rbf)
print("Sigmoid kernel:", sc_svc_sig)


Linear kernel: 0.9224160107012399
Polynomial kernel: 0.9222616658949426
Radio Base Frequency kernel: 0.9240109070329784
Sigmoid kernel: 0.8744662242115553


**RandomizedSearchCV**

In [425]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

dists = {
    "C": uniform(loc=0, scale=3),
    "gamma": uniform(loc=0,scale=(3/171)) # number of features
}

linear_cv = RandomizedSearchCV(
    svm.LinearSVC(random_state=12), 
    {"C": uniform(loc=0, scale=3),"penalty":['l1','l2']},
    n_iter = 4,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=12,
    verbose=2)

poly_cv = RandomizedSearchCV(
    svm.SVC(random_state=12,kernel='poly'), 
    dists,
    n_iter = 4,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=12,
    verbose=2)

rbf_cv = RandomizedSearchCV(
    svm.SVC(random_state=13,kernel='rbf'), 
    dists,
    n_iter = 4,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=14,
    verbose=2)

sig_cv = RandomizedSearchCV(
    svm.SVC(random_state=15,kernel='sigmoid'), 
    dists,
    n_iter = 4,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=12,
    verbose=2)

In [168]:
linear_cv.fit(imp_x_train,imp_y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ...................C=0.4624885271390171, penalty=l1; total time=   0.0s
[CV] END ...................C=0.4624885271390171, penalty=l1; total time=   0.0s
[CV] END ...................C=0.4624885271390171, penalty=l1; total time=   0.0s
[CV] END ...................C=0.4624885271390171, penalty=l1; total time=   0.0s
[CV] END ...................C=0.4624885271390171, penalty=l1; total time=   0.0s
[CV] END ...................C=2.6201614572848753, penalty=l1; total time=   0.0s
[CV] END ...................C=2.6201614572848753, penalty=l1; total time=   0.0s
[CV] END ...................C=2.6201614572848753, penalty=l1; total time=   0.0s
[CV] END ...................C=2.6201614572848753, penalty=l1; total time=   0.0s
[CV] END ...................C=2.6201614572848753, penalty=l1; total time=   0.0s
[CV] END ...................C=1.6012181801408931, penalty=l1; total time=   0.0s
[CV] END ...................C=1.6012181801408931,

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END ...................C=1.0514348280149775, penalty=l2; total time=  22.5s
[CV] END ...................C=1.0514348280149775, penalty=l2; total time=  22.6s
[CV] END ...................C=1.0514348280149775, penalty=l2; total time=  22.6s
[CV] END ...................C=1.0514348280149775, penalty=l2; total time=  22.6s


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END ...................C=1.0514348280149775, penalty=l2; total time=  22.7s


RandomizedSearchCV(cv=5, estimator=LinearSVC(random_state=12), n_iter=4,
                   n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x177796f10>,
                                        'penalty': ['l1', 'l2']},
                   random_state=12, refit='accuracy',
                   scoring=['accuracy', 'f1', 'precision', 'recall'],
                   verbose=2)

In [169]:
lin_cv_pred = linear_cv.predict(imp_x_test)
lin_cv_sc = linear_cv.score(imp_x_test,imp_y_test)
lin_cv_results = pd.DataFrame(linear_cv.cv_results_)

In [170]:
poly_cv.fit(imp_x_train,imp_y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END .....C=0.789945045555404, gamma=0.00936384900667189; total time= 8.8min
[CV] END .....C=0.789945045555404, gamma=0.00936384900667189; total time= 9.1min
[CV] END .....C=0.789945045555404, gamma=0.00936384900667189; total time= 9.3min
[CV] END .....C=0.789945045555404, gamma=0.00936384900667189; total time= 9.6min
[CV] END ...C=0.4624885271390171, gamma=0.012983328009042188; total time= 9.6min
[CV] END ...C=0.4624885271390171, gamma=0.012983328009042188; total time=10.0min
[CV] END .....C=0.789945045555404, gamma=0.00936384900667189; total time=10.2min
[CV] END ...C=0.4624885271390171, gamma=0.012983328009042188; total time=10.2min
[CV] END ...C=0.4624885271390171, gamma=0.012983328009042188; total time=10.3min
[CV] END ..C=2.7021445623510365, gamma=0.0005863408355499051; total time=  58.1s
[CV] END ..C=2.7021445623510365, gamma=0.0005863408355499051; total time=  58.6s
[CV] END ..C=2.7021445623510365, gamma=0.00058634

RandomizedSearchCV(cv=5, estimator=SVC(kernel='poly', random_state=12),
                   n_iter=4, n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x177786b10>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x177795410>},
                   random_state=12, refit='accuracy',
                   scoring=['accuracy', 'f1', 'precision', 'recall'],
                   verbose=2)

In [171]:
poly_cv_pred = poly_cv.predict(imp_x_test)
poly_cv_sc = poly_cv.score(imp_x_test,imp_y_test)
poly_cv_results = pd.DataFrame(poly_cv.cv_results_)

In [ ]:
rbf_cv.fit(imp_x_train,imp_y_train)

In [ ]:
rbf_cv_pred = rbf_cv.predict(imp_x_test)
rbf_cv_sc = rbf_cv.score(imp_x_test,imp_y_test)
rbf_cv_results = pd.DataFrame(rbf_cv.cv_results_)

In [ ]:
rbf_cv_results[['mean_test_accuracy','std_test_accuracy',
       'rank_test_accuracy','mean_test_f1',
       'std_test_f1', 'rank_test_f1','mean_test_precision',
       'std_test_precision', 'rank_test_precision','mean_test_recall', 'std_test_recall',
       'rank_test_recall']]

,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,mean_test_f1,std_test_f1,rank_test_f1,mean_test_precision,std_test_precision,rank_test_precision,mean_test_recall,std_test_recall,rank_test_recall
0,0.913612,0.000031,10,0.000000,0.000000,10,0.000000,0.000000,10,0.000000,0.000000,10
1,0.927787,0.000917,1,0.357237,0.012982,1,0.773125,0.017703,5,0.232417,0.010728,1
2,0.916587,0.000626,9,0.087810,0.012314,9,0.792139,0.036353,3,0.046519,0.006826,9
3,0.922351,0.000425,5,0.254947,0.004447,5,0.745749,0.020032,7,0.153805,0.003620,5
4,0.921183,0.000360,6,0.190872,0.007985,7,0.843530,0.011045,1,0.107645,0.005121,7
5,0.920482,0.000351,7,0.215655,0.006546,6,0.729780,0.018134,8,0.126579,0.004818,6
6,0.927257,0.000653,2,0.354514,0.009872,2,0.759940,0.018979,6,0.231337,0.009015,2
7,0.919407,0.000578,8,0.167206,0.014116,8,0.780049,0.025592,4,0.093762,0.009058,8
8,0.925871,0.000702,3,0.308916,0.010620,3,0.793536,0.014884,2,0.191849,0.007881,3
9,0.922616,0.000673,4,0.270649,0.008281,4,0.728982,0.021667,9,0.166248,0.006235,4


In [174]:
sig_cv.fit(imp_x_train,imp_y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ...C=0.4624885271390171, gamma=0.012983328009042188; total time=16.0min
[CV] END .....C=0.789945045555404, gamma=0.00936384900667189; total time=16.0min
[CV] END .....C=0.789945045555404, gamma=0.00936384900667189; total time=16.0min
[CV] END ...C=0.4624885271390171, gamma=0.012983328009042188; total time=16.1min
[CV] END ...C=0.4624885271390171, gamma=0.012983328009042188; total time=16.1min
[CV] END ...C=0.4624885271390171, gamma=0.012983328009042188; total time=16.3min
[CV] END ...C=0.4624885271390171, gamma=0.012983328009042188; total time=16.6min
[CV] END .....C=0.789945045555404, gamma=0.00936384900667189; total time=32.6min
[CV] END ..C=0.04372488745625902, gamma=0.016118368563155877; total time=16.7min
[CV] END ..C=0.04372488745625902, gamma=0.016118368563155877; total time=16.7min
[CV] END .....C=0.789945045555404, gamma=0.00936384900667189; total time=32.7min
[CV] END ..C=0.04372488745625902, gamma=0.0161183

RandomizedSearchCV(cv=5, estimator=SVC(kernel='sigmoid', random_state=15),
                   n_iter=4, n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x177786b10>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x177795410>},
                   random_state=12, refit='accuracy',
                   scoring=['accuracy', 'f1', 'precision', 'recall'],
                   verbose=2)

In [175]:
sig_cv_pred = sig_cv.predict(imp_x_test)
sig_cv_sc = sig_cv.score(imp_x_test,imp_y_test)
sig_cv_results = pd.DataFrame(sig_cv.cv_results_)

**###### CV RESULTS ######**

In [182]:
print("Linear kernel w/ CV:", lin_cv_sc)
lin_cv_results


Linear kernel w/ CV: 0.9223131141637084


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,...,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall
0,0.030054,0.001364,0.00000,0.000000,0.462489,l1,"{'C': 0.4624885271390171, 'penalty': 'l1'}",NaN,NaN,NaN,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,0.034010,0.001043,0.00000,0.000000,2.620161,l1,"{'C': 2.6201614572848753, 'penalty': 'l1'}",NaN,NaN,NaN,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,0.034519,0.000390,0.00000,0.000000,1.601218,l1,"{'C': 1.6012181801408931, 'penalty': 'l1'}",NaN,NaN,NaN,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,22.591306,0.091634,0.00681,0.000188,1.051435,l2,"{'C': 1.0514348280149775, 'penalty': 'l2'}",0.919193,0.920626,0.923264,...,0.034979,1,0.221951,0.208283,0.271951,0.254878,0.208537,0.23312,0.025796,1


In [183]:

print("Polynomial kernel w/ CV:", poly_cv_sc)
poly_cv_results


Polynomial kernel w/ CV: 0.9222616658949426


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,...,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall
0,615.391936,47.200343,8.067477,1.137256,0.462489,0.012983,"{'C': 0.4624885271390171, 'gamma': 0.012983328...",0.914673,0.912138,0.914112,...,0.021108,4,0.317073,0.311815,0.330488,0.346341,0.320732,0.325290,0.012166,1
1,555.826890,26.990711,7.689679,0.568253,0.789945,0.009364,"{'C': 0.789945045555404, 'gamma': 0.0093638490...",0.916217,0.915335,0.917310,...,0.016450,3,0.309756,0.310597,0.328049,0.318293,0.297561,0.312851,0.010090,2
2,266.792503,35.574889,4.868188,0.077565,0.043725,0.016118,"{'C': 0.04372488745625902, 'gamma': 0.01611836...",0.920295,0.920847,0.923594,...,0.016654,2,0.256098,0.257004,0.287805,0.274390,0.259756,0.267010,0.012318,3
3,49.771353,3.922140,6.032902,0.818657,2.702145,0.000586,"{'C': 2.7021445623510365, 'gamma': 0.000586340...",0.912358,0.913350,0.913671,...,0.031090,1,0.036585,0.045067,0.050000,0.045122,0.047561,0.044867,0.004523,4


In [184]:

print("Radio Base Frequency kernel w/ CV:", rbf_cv_sc)
rbf_cv_results


Radio Base Frequency kernel w/ CV: 0.9248340793332305


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,...,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall
0,3349.170865,2.077310,970.412805,0.034132,1.54183,0.013564,"{'C': 1.5418300313180708, 'gamma': 0.013564299...",0.915224,0.917319,0.919846,...,0.052228,4,0.139024,0.153471,0.174390,0.176829,0.173171,0.163377,0.014760,2
1,49.492845,0.617307,9.452840,0.065113,2.611283,0.000141,"{'C': 2.611283057174438, 'gamma': 0.0001411745...",0.915886,0.916657,0.917089,...,0.035614,1,0.087805,0.088916,0.093902,0.109756,0.092683,0.094612,0.007904,3
2,4262.759614,1.706641,21.560587,0.150022,0.929208,0.0168,"{'C': 0.9292077765156124, 'gamma': 0.016800065...",0.911035,0.914122,0.912900,...,0.086423,3,0.042683,0.065773,0.057317,0.069512,0.054878,0.058033,0.009357,4
3,2102.070394,2.113401,11.411021,0.125519,1.53935,0.005584,"{'C': 1.5393501368036615, 'gamma': 0.005583937...",0.921398,0.923382,0.925579,...,0.035792,2,0.220732,0.227771,0.262195,0.265854,0.245122,0.244335,0.017967,1


In [185]:

print("Sigmoid kernel w/ CV:", sig_cv_sc)
sig_cv_results

Sigmoid kernel w/ CV: 0.919432011112826


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,...,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall
0,967.531438,12.885039,4.632030,0.206219,0.462489,0.012983,"{'C': 0.4624885271390171, 'gamma': 0.012983328...",0.863080,0.855584,0.865821,...,0.038038,3,0.206098,0.208283,0.274390,0.295122,0.215854,0.239949,0.037308,2
1,1365.496874,486.026753,196.336007,383.034714,0.789945,0.009364,"{'C': 0.789945045555404, 'gamma': 0.0093638490...",0.867380,0.860986,0.867916,...,0.008934,2,0.229268,0.238733,0.241463,0.258537,0.245122,0.242625,0.009532,1
2,1002.430255,8.838170,4.989593,0.562988,0.043725,0.016118,"{'C': 0.04372488745625902, 'gamma': 0.01611836...",0.868372,0.861096,0.865160,...,0.004098,4,0.163415,0.177832,0.175610,0.180488,0.175610,0.174591,0.005870,3
3,419.339836,471.357997,3.832931,0.117366,2.702145,0.000586,"{'C': 2.7021445623510365, 'gamma': 0.000586340...",0.917539,0.919965,0.919184,...,0.025505,1,0.153659,0.177832,0.175610,0.185366,0.164634,0.171420,0.011088,4


**The 17 most important features from PCA etc. EXCLUDING apache**

* d1_lactate_max
* d1_lactate_min
* h1_lactate_min
* h1_lactate_max
* d1_sysbp_invasive_min
* h1_albumin_min
* h1_albumin_max
* d1_mbp_invasive_min
* d1_inr_max
* h1_inr_max
* d1_sysbp_min
* d1_sysbp_noninvasive_min
* d1_bun_max
* d1_albumin_min
* d1_inr_min
* h1_inr_min
* d1_bun_min

In [69]:
best_feats = ['d1_lactate_max','d1_lactate_min','h1_lactate_min','h1_lactate_max','d1_sysbp_invasive_min','h1_albumin_min','h1_albumin_max','d1_mbp_invasive_min','d1_inr_max','h1_inr_max','d1_sysbp_min','d1_sysbp_noninvasive_min','d1_bun_max','d1_albumin_min','d1_inr_min','h1_inr_min','d1_bun_min']


Testing best feats with regular SVC

In [150]:
svc_poly2 = svm.SVC(kernel='poly')
svc_rbf2 = svm.SVC(kernel='rbf')
svc_sig2 = svm.SVC(kernel='sigmoid')
svc_lin2 = svm.LinearSVC()

In [151]:
imp_x_train_best = imp_x_train.filter(best_feats)
imp_x_test_best = imp_x_test.filter(best_feats)
### Note: using same divides as in the previous more general model

In [152]:
svc_lin2.fit(imp_x_train_best,imp_y_train)
svc_lin_pred2 = svc_lin2.predict(imp_x_test_best)
sc_svc_lin2 = svc_lin2.score(imp_x_test_best,imp_y_test)

In [153]:
svc_poly2.fit(imp_x_train_best,imp_y_train)
svc_pol_pred2 = svc_poly2.predict(imp_x_test_best)
sc_svc_pol2 = svc_poly2.score(imp_x_test_best,imp_y_test)

In [154]:
svc_rbf2.fit(imp_x_train_best,imp_y_train)
svc_rbf_pred2 = svc_rbf2.predict(imp_x_test_best)
sc_svc_rbf2 = svc_rbf2.score(imp_x_test_best,imp_y_test)

In [155]:
svc_sig2.fit(imp_x_train_best,imp_y_train)
svc_sig_pred2 = svc_sig2.predict(imp_x_test_best)
sc_svc_sig2 = svc_sig2.score(imp_x_test_best,imp_y_test)

In [156]:
print("Linear kernel:", sc_svc_lin2)
print("Polynomial kernel:", sc_svc_pol2)
print("Radio Base Frequency kernel:", sc_svc_rbf2)
print("Sigmoid kernel:", sc_svc_sig2)


Linear kernel: 0.9156248392241602
Polynomial kernel: 0.914801666923908
Radio Base Frequency kernel: 0.9161907701805834
Sigmoid kernel: 0.8438545042959305


Testing best feats with RandomizedCV SVC

In [208]:
lin_cv2 = RandomizedSearchCV(
    svm.LinearSVC(random_state=12), 
    {"C": uniform(loc=0, scale=3),"penalty":['l1','l2']},
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=12,
    verbose=0)

poly_cv2 = RandomizedSearchCV(
    svm.SVC(random_state=12,kernel='poly'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=12,
    verbose=0)

rbf_cv2 = RandomizedSearchCV(
    svm.SVC(random_state=13,kernel='rbf'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=14,
    verbose=0)

sig_cv2 = RandomizedSearchCV(
    svm.SVC(random_state=15,kernel='sigmoid'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=12,
    verbose=0)

In [209]:
lin_cv2.fit(imp_x_train_best,imp_y_train)
lin_cv_pred2 = lin_cv2.predict(imp_x_test_best)
lin_cv_sc2 = lin_cv2.score(imp_x_test_best,imp_y_test)
lin_cv_results2 = pd.DataFrame(lin_cv2.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increas

In [210]:
poly_cv2.fit(imp_x_train_best,imp_y_train)
poly_cv_pred2 = poly_cv2.predict(imp_x_test_best)
poly_cv_sc2 = poly_cv2.score(imp_x_test_best,imp_y_test)
poly_cv_results2 = pd.DataFrame(poly_cv2.cv_results_)

In [211]:
rbf_cv2.fit(imp_x_train_best,imp_y_train)
rbf_cv_pred2 = rbf_cv2.predict(imp_x_test_best)
rbf_cv_sc2 = rbf_cv2.score(imp_x_test_best,imp_y_test)
rbf_cv_results2 = pd.DataFrame(rbf_cv2.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Librar

In [212]:
sig_cv2.fit(imp_x_train_best,imp_y_train)
sig_cv_pred2 = sig_cv2.predict(imp_x_test_best)
sig_cv_sc2 = sig_cv2.score(imp_x_test_best,imp_y_test)
sig_cv_results2 = pd.DataFrame(sig_cv2.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Librar

In [214]:
print("Linear CV kernel:", lin_cv_sc2)
print("Polynomial CV kernel:", poly_cv_sc2)
print("Radio Base Frequency CV kernel:", rbf_cv_sc2)
print("Sigmoid CV kernel:", sig_cv_sc2)

Linear CV kernel: 0.9153675978803313
Polynomial CV kernel: 0.9148531151926738
Radio Base Frequency CV kernel: 0.9159849771055204
Sigmoid CV kernel: 0.9089365642846118


**Testing on unlabeled testing data**
**Had to use .fillna(0) because otherwise SVC will not accept NaN values. However, this could definitely mess things up as the 0 values for certain features may cause issues.**

In [387]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'most_frequent') # another: median
test_trunc_x_imp = imputer.fit_transform(testing_X)

In [388]:
svc_lin_test_pred = svc_lin.predict(test_trunc_x_imp)

In [389]:
svc_poly_test_pred = svc_poly.predict(test_trunc_x_imp)

In [390]:
svc_rbf_test_pred = svc_rbf.predict(test_trunc_x_imp)

In [391]:
svc_sig_test_pred = svc_sig.predict(test_trunc_x_imp)

In [196]:
print("Linear kernel:", svc_lin_test_sc)
print("Polynomial kernel:", svc_poly_test_sc)
print("Radio Base Frequency kernel:", svc_rbf_test_sc)
print("Sigmoid kernel:", svc_sig_test_sc)

Linear kernel: 0.9995166378345375
Polynomial kernel: 0.49363997150707234
Radio Base Frequency kernel: 1.0
Sigmoid kernel: 0.9955734201689224


In [177]:
lin_cv_pred_test = linear_cv.predict(testing_X.fillna(0))
lin_cv_results_test = pd.DataFrame(linear_cv.cv_results_)

In [178]:
poly_cv_pred_test = poly_cv.predict(testing_X.fillna(0))
poly_cv_results_test = pd.DataFrame(poly_cv.cv_results_)

In [179]:
rbf_cv_pred_test = rbf_cv.predict(testing_X.fillna(0))
rbf_cv_results_test = pd.DataFrame(rbf_cv.cv_results_)

In [180]:
sig_cv_pred_test = sig_cv.predict(testing_X.fillna(0))
sig_cv_results_test = pd.DataFrame(sig_cv.cv_results_)

In [215]:
print("Linear kernel w/ CV:", lin_cv_sc_test)
print("Polynomial kernel w/ CV:", poly_cv_sc_test)
print("Radio Base Frequency kernel w/ CV:", rbf_cv_sc_test)
print("Sigmoid kernel w/ CV:", sig_cv_sc_test)

Linear kernel w/ CV: 0.9993639971507072
Polynomial kernel w/ CV: 0.3835605983514806
Radio Base Frequency kernel w/ CV: 1.0
Sigmoid kernel w/ CV: 0.9980156711102066


In [372]:
lin_cv_pred_test = linear_cv.predict(test_trunc_x_imp)
lin_cv_results_test = pd.DataFrame(linear_cv.cv_results_)

In [373]:
poly_cv_pred_test = poly_cv.predict(test_trunc_x_imp)
poly_cv_results_test = pd.DataFrame(poly_cv.cv_results_)

In [374]:
rbf_cv_pred_test = rbf_cv.predict(test_trunc_x_imp)
rbf_cv_results_test = pd.DataFrame(rbf_cv.cv_results_)

In [375]:
sig_cv_pred_test = sig_cv.predict(test_trunc_x_imp)
sig_cv_results_test = pd.DataFrame(sig_cv.cv_results_)

In [220]:
print("Linear CV kernel:", lin_cv_sc_test)
print("Polynomial CV kernel:", poly_cv_sc_test)
print("Radio Base Frequency CV kernel:", rbf_cv_sc_test)
print("Sigmoid CV kernel:", sig_cv_sc_test)

Linear CV kernel: 0.9999745598860282
Polynomial CV kernel: 0.4933092500254401
Radio Base Frequency CV kernel: 1.0
Sigmoid CV kernel: 1.0


**#################################### BREAK ####################################**

***SMOTE***

In [287]:
smote_X = smote.drop("hospital_death",axis=1)
smote_y = smote['hospital_death']
random.seed(101)
smote_x_train, smote_x_test, smote_y_train, smote_y_test = train_test_split(smote_X,smote_y,test_size=.3, random_state=47)

In [307]:
smote_x_train.columns

Index(['age', 'bmi', 'height', 'pre_icu_los_days', 'weight', 'albumin_apache',
       'apache_2_diagnosis', 'apache_3j_diagnosis', 'arf_apache',
       'bilirubin_apache',
       ...
       'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob', 'aids',
       'cirrhosis', 'diabetes_mellitus', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis'],
      dtype='object', length=170)

In [308]:
svc_poly = svm.SVC(kernel='poly')
svc_rbf = svm.SVC(kernel='rbf')
svc_sig = svm.SVC(kernel='sigmoid')
svc_lin = svm.LinearSVC()

In [309]:
svc_lin.fit(smote_x_train,smote_y_train)
svc_lin_pred = svc_lin.predict(smote_x_test)
sc_svc_lin = svc_lin.score(smote_x_test,smote_y_test)

In [305]:
svc_lin.feature_names_in_

array(['age', 'bmi', 'height', 'pre_icu_los_days', 'weight',
       'albumin_apache', 'apache_2_diagnosis', 'apache_3j_diagnosis',
       'arf_apache', 'bilirubin_apache', 'bun_apache',
       'creatinine_apache', 'fio2_apache', 'gcs_eyes_apache',
       'gcs_motor_apache', 'gcs_unable_apache', 'gcs_verbal_apache',
       'glucose_apache', 'heart_rate_apache', 'hematocrit_apache',
       'intubated_apache', 'map_apache', 'paco2_apache',
       'paco2_for_ph_apache', 'pao2_apache', 'ph_apache',
       'resprate_apache', 'sodium_apache', 'temp_apache',
       'urineoutput_apache', 'ventilated_apache', 'wbc_apache',
       'd1_diasbp_invasive_max', 'd1_diasbp_invasive_min',
       'd1_diasbp_max', 'd1_diasbp_min', 'd1_diasbp_noninvasive_max',
       'd1_diasbp_noninvasive_min', 'd1_heartrate_max',
       'd1_heartrate_min', 'd1_mbp_invasive_max', 'd1_mbp_invasive_min',
       'd1_mbp_max', 'd1_mbp_min', 'd1_mbp_noninvasive_max',
       'd1_mbp_noninvasive_min', 'd1_resprate_max', 'd1_resp

In [312]:
svc_poly.fit(smote_x_train,smote_y_train)
svc_poly_pred = svc_poly.predict(smote_x_test)
sc_svc_poly = svc_poly.score(smote_x_test,smote_y_test)

In [313]:
svc_rbf.fit(smote_x_train,smote_y_train)
svc_rbf_pred = svc_rbf.predict(smote_x_test)
sc_svc_rbf = svc_rbf.score(smote_x_test,smote_y_test)

In [314]:
svc_sig.fit(smote_x_train,smote_y_train)
svc_sig_pred = svc_sig.predict(smote_x_test)
sc_svc_sig = svc_sig.score(smote_x_test,smote_y_test)

Results

In [315]:
print("Linear kernel:", sc_svc_lin)
print("Polynomial kernel:", sc_svc_pol)
print("Radio Base Frequency kernel:", sc_svc_rbf)
print("Sigmoid kernel:", sc_svc_sig)


Linear kernel: 0.8057544417926279
Polynomial kernel: 0.9222616658949426
Radio Base Frequency kernel: 0.8167594802439672
Sigmoid kernel: 0.7260673561389552


**RandomCV**

In [298]:
dists = {
    "C": uniform(loc=0, scale=3),
    "gamma": uniform(loc=0,scale=(3/171)) # number of features
}

lin_cv = RandomizedSearchCV(
    svm.LinearSVC(random_state=12), 
    {"C": uniform(loc=0, scale=3),"penalty":['l1','l2']},
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=114,
    verbose=0)

poly_cv = RandomizedSearchCV(
    svm.SVC(random_state=12,kernel='poly'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=115,
    verbose=0)

rbf_cv = RandomizedSearchCV(
    svm.SVC(random_state=13,kernel='rbf'), 
    dists,
    n_iter = 110,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=116,
    verbose=0)

sig_cv = RandomizedSearchCV(
    svm.SVC(random_state=15,kernel='sigmoid'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=117,
    verbose=0)

In [299]:
lin_cv.fit(smote_x_train,smote_y_train)
lin_cv_pred = lin_cv.predict(smote_x_test)
lin_cv_sc = lin_cv.score(smote_x_test,smote_y_test)
lin_cv_results = pd.DataFrame(lin_cv.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increas

In [300]:
poly_cv.fit(smote_x_train,smote_y_train)
poly_cv_pred = poly_cv.predict(smote_x_test)
poly_cv_sc = poly_cv.score(smote_x_test,smote_y_test)
poly_cv_results = pd.DataFrame(poly_cv.cv_results_)

In [301]:
rbf_cv.fit(smote_x_train,smote_y_train)
rbf_cv_pred = rbf_cv.predict(smote_x_test)
rbf_cv_sc = rbf_cv.score(smote_x_test,smote_y_test)
rbf_cv_results = pd.DataFrame(rbf_cv.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Librar

In [424]:
rbf_cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,...,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall
0,650.712089,6.969153,36.635373,0.210464,0.106767,0.013678,"{'C': 0.10676710492889818, 'gamma': 0.01367797...",0.913629,0.913629,0.913629,...,0.000000,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10
1,391.299244,5.128305,23.364456,0.207180,2.056923,0.003705,"{'C': 2.0569231285180263, 'gamma': 0.003704949...",0.927181,0.927960,0.926636,...,0.017703,5,0.239856,0.229937,0.218215,0.248649,0.225428,0.232417,0.010728,1
2,3913.884127,458.608359,591.594798,448.587815,0.724986,0.01657,"{'C': 0.7249856571992477, 'gamma': 0.016569637...",0.917757,0.916277,0.916277,...,0.036353,3,0.056808,0.042381,0.042381,0.052252,0.038774,0.046519,0.006826,9
3,4456.220874,7.645968,98.958671,1.658096,1.244441,0.013076,"{'C': 1.2444409071927485, 'gamma': 0.013076094...",0.922118,0.922040,0.922975,...,0.020032,7,0.160505,0.153291,0.152390,0.149550,0.153291,0.153805,0.003620,5
4,501.619941,27.277372,84.988784,0.873781,0.402729,0.00743,"{'C': 0.4027290119076675, 'gamma': 0.007430481...",0.921651,0.921028,0.921573,...,0.011045,1,0.111812,0.104599,0.114518,0.107207,0.100090,0.107645,0.005121,7
5,5903.903042,451.975264,986.823764,37.153282,1.340712,0.015277,"{'C': 1.34071229863613, 'gamma': 0.01527705965...",0.920327,0.920327,0.921028,...,0.018134,8,0.135257,0.124436,0.127142,0.120721,0.125338,0.126579,0.004818,6
6,963.555064,14.730845,81.415162,10.265329,1.53282,0.005997,"{'C': 1.5328202292244688, 'gamma': 0.005997140...",0.927025,0.927648,0.926168,...,0.018979,6,0.245266,0.229035,0.221821,0.237838,0.222723,0.231337,0.009015,2
7,8733.772651,26.155030,454.169053,449.505748,0.889659,0.014046,"{'C': 0.8896585620117006, 'gamma': 0.014045534...",0.920405,0.919315,0.919470,...,0.025592,4,0.110911,0.091073,0.091975,0.090991,0.083859,0.093762,0.009058,8
8,1807.235928,478.174613,39.792777,5.191415,2.010891,0.001823,"{'C': 2.010891115338811, 'gamma': 0.0018231769...",0.926090,0.926246,0.925156,...,0.014884,2,0.201082,0.193868,0.186655,0.198198,0.179441,0.191849,0.007881,3
9,8869.047802,14.876884,31.556014,0.562615,1.440763,0.013151,"{'C': 1.4407625884521686, 'gamma': 0.013151306...",0.922196,0.922664,0.923287,...,0.021667,9,0.174031,0.168620,0.166817,0.154955,0.166817,0.166248,0.006235,4


In [302]:
sig_cv.fit(smote_x_train,smote_y_train)
sig_cv_pred = sig_cv.predict(smote_x_test)
sig_cv_sc = sig_cv.score(smote_x_test,smote_y_test)
sig_cv_results = pd.DataFrame(sig_cv.cv_results_)

In [411]:
rbf_cv.score
#rbf_cv_results[['mean_test_f1','std_test_f1', 'rank_test_f1','mean_test_precision','std_test_precision', 'rank_test_precision','mean_test_recall', 'std_test_recall','rank_test_recall']]

<bound method BaseSearchCV.score of RandomizedSearchCV(cv=5, estimator=SVC(random_state=13), n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x175605c90>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x1779944d0>},
                   random_state=216, refit='accuracy',
                   scoring=['accuracy', 'f1', 'precision', 'recall'])>

Random CV results

In [396]:
print("Linear CV kernel:", lin_cv_sc)
print("Polynomial CV kernel:", poly_cv_sc)
print("Radio Base Frequency CV kernel:", rbf_cv_sc)
print("Sigmoid CV kernel:", sig_cv_sc)

Linear CV kernel: 0.9269826270262412
Polynomial CV kernel: 0.9263284146252817
Radio Base Frequency CV kernel: 0.9301809987642654
Sigmoid CV kernel: 0.9049574761939376


17 best to rerun

In [264]:
smote_x_train_best = smote_x_train.filter(best_feats)
smote_x_test_best = smote_x_test.filter(best_feats)

17 best regular SVC

In [265]:
svc_poly2 = svm.SVC(kernel='poly')
svc_rbf2 = svm.SVC(kernel='rbf')
svc_sig2 = svm.SVC(kernel='sigmoid')
svc_lin2 = svm.LinearSVC()

In [266]:
svc_lin2.fit(smote_x_train_best,smote_y_train)
svc_lin_pred2 = svc_lin2.predict(smote_x_test_best)
sc_svc_lin2 = svc_lin2.score(smote_x_test_best,smote_y_test)

In [267]:
svc_poly2.fit(smote_x_train_best,smote_y_train)
svc_poly_pred2 = svc_poly2.predict(smote_x_test_best)
sc_svc_poly2 = svc_poly2.score(smote_x_test_best,smote_y_test)

In [268]:
svc_rbf2.fit(smote_x_train_best,smote_y_train)
svc_rbf_pred2 = svc_rbf2.predict(smote_x_test_best)
sc_svc_rbf2 = svc_rbf2.score(smote_x_test_best,smote_y_test)

In [269]:
svc_sig2.fit(smote_x_train_best,smote_y_train)
svc_sig_pred2 = svc_sig2.predict(smote_x_test_best)
sc_svc_sig2 = svc_sig2.score(smote_x_test_best,smote_y_test)

In [270]:
print("Linear kernel:", sc_svc_lin2)
print("Polynomial kernel:", sc_svc_pol2)
print("Radio Base Frequency kernel:", sc_svc_rbf2)
print("Sigmoid kernel:", sc_svc_sig2)


Linear kernel: 0.7423760275788915
Polynomial kernel: 0.914801666923908
Radio Base Frequency kernel: 0.7536462476796606
Sigmoid kernel: 0.611243701935826


SVC RandomCV Run best 17

In [271]:
dists = {
    "C": uniform(loc=0, scale=3),
    "gamma": uniform(loc=0,scale=(3/171)) # number of features
}

lin_cv2 = RandomizedSearchCV(
    svm.LinearSVC(random_state=12), 
    {"C": uniform(loc=0, scale=3),"penalty":['l1','l2']},
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=104,
    verbose=0)

poly_cv2 = RandomizedSearchCV(
    svm.SVC(random_state=12,kernel='poly'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=105,
    verbose=0)

rbf_cv2 = RandomizedSearchCV(
    svm.SVC(random_state=13,kernel='rbf'), 
    dists,
    n_iter = 110,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=106,
    verbose=0)

sig_cv2 = RandomizedSearchCV(
    svm.SVC(random_state=15,kernel='sigmoid'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=107,
    verbose=0)

In [272]:
lin_cv2.fit(smote_x_train_best,smote_y_train)
lin_cv_pred2 = lin_cv2.predict(smote_x_test_best)
lin_cv_sc2 = lin_cv2.score(smote_x_test_best,smote_y_test)
lin_cv_results2 = pd.DataFrame(lin_cv2.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increas

In [273]:
poly_cv2.fit(smote_x_train_best,smote_y_train)
poly_cv_pred2 = poly_cv2.predict(smote_x_test_best)
poly_cv_sc2 = poly_cv2.score(smote_x_test_best,smote_y_test)
poly_cv_results2 = pd.DataFrame(poly_cv2.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Librar

In [274]:
rbf_cv2.fit(smote_x_train_best,smote_y_train)
rbf_cv_pred2 = rbf_cv2.predict(smote_x_test_best)
rbf_cv_sc2 = rbf_cv2.score(smote_x_test_best,smote_y_test)
rbf_cv_results2 = pd.DataFrame(rbf_cv2.cv_results_)

In [275]:
sig_cv2.fit(smote_x_train_best,smote_y_train)
sig_cv_pred2 = sig_cv2.predict(smote_x_test_best)
sig_cv_sc2 = sig_cv2.score(smote_x_test_best,smote_y_test)
sig_cv_results2 = pd.DataFrame(sig_cv2.cv_results_)

**randomCV results**

In [276]:
print("Linear CV kernel:", lin_cv_sc2)
print("Polynomial CV kernel:", poly_cv_sc2)
print("Radio Base Frequency CV kernel:", rbf_cv_sc2)
print("Sigmoid CV kernel:", sig_cv_sc2)

Linear CV kernel: 0.7422434367541766
Polynomial CV kernel: 0.7277910368602493
Radio Base Frequency CV kernel: 0.7504640678865022
Sigmoid CV kernel: 0.7360116679925749


***The real test***

Regular SVC

In [304]:
test_trunc_x.columns

Index(['age', 'bmi', 'height', 'pre_icu_los_days', 'weight', 'albumin_apache',
       'apache_2_diagnosis', 'apache_3j_diagnosis', 'arf_apache',
       'bilirubin_apache',
       ...
       'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob', 'aids',
       'cirrhosis', 'diabetes_mellitus', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis'],
      dtype='object', length=170)

In [306]:
smote_x_train.columns

Index(['age', 'bmi', 'height', 'pre_icu_los_days', 'weight', 'albumin_apache',
       'apache_2_diagnosis', 'apache_3j_diagnosis', 'arf_apache',
       'bilirubin_apache',
       ...
       'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob', 'aids',
       'cirrhosis', 'diabetes_mellitus', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis'],
      dtype='object', length=170)

In [ ]:
svc_lin_test_pred = svc_lin.predict(testing_X.fillna(0))

In [ ]:
svc_poly_test_pred = svc_poly.predict(testing_X.fillna(0))

In [ ]:
svc_rbf_test_pred = svc_rbf.predict(testing_X.fillna(0))

In [ ]:
svc_sig_test_pred = svc_sig.predict(testing_X.fillna(0))

In [319]:
# print("Linear kernel:", svc_lin_test_sc)
# print("Polynomial kernel:", svc_poly_test_sc)
# print("Radio Base Frequency kernel:", svc_rbf_test_sc)
# print("Sigmoid kernel:", svc_sig_test_sc)

Linear kernel: 0.998499033275669
Polynomial kernel: 0.5849445405515417
Radio Base Frequency kernel: 2.5440113971710593e-05
Sigmoid kernel: 0.972753637936298


RandomizedCV

In [ ]:
lin_cv_pred_test = lin_cv.predict(testing_X.fillna(0))
lin_cv_results_test = pd.DataFrame(lin_cv.cv_results_)

In [ ]:
poly_cv_pred_test = poly_cv.predict(testing_X.fillna(0))
poly_cv_results_test = pd.DataFrame(poly_cv.cv_results_)

In [ ]:
rbf_cv_pred_test = rbf_cv.predict(testing_X.fillna(0))
rbf_cv_results_test = pd.DataFrame(rbf_cv.cv_results_)

In [ ]:
sig_cv_pred_test = sig_cv.predict(testing_X.fillna(0))
sig_cv_results_test = pd.DataFrame(sig_cv.cv_results_)

In [324]:
# print("Linear CV kernel:", lin_cv_sc_test)
# print("Polynomial CV kernel:", poly_cv_sc_test)
# print("Radio Base Frequency CV kernel", rbf_cv_sc_test)
# print("Sigmoid CV kernel:", sig_cv_sc_test)

Linear CV kernel: 0.9994148773786506
Polynomial CV kernel: 0.5346748753434415
Radio Base Frequency CV kernel 2.5440113971710593e-05
Sigmoid CV kernel: 0.43655235575455376


**#################################### BREAK ####################################**

*Missing = 0 set*

**SVC, normal and best 17**

In [326]:
miss_X = miss.drop("hospital_death",axis=1)
miss_y = miss['hospital_death']
random.seed(10)
miss_x_train, miss_x_test, miss_y_train, miss_y_test = train_test_split(miss_X,miss_y,test_size=.3, random_state=23)

In [327]:
miss_X.columns

Index(['age', 'bmi', 'height', 'pre_icu_los_days', 'weight', 'albumin_apache',
       'apache_2_diagnosis', 'apache_3j_diagnosis', 'arf_apache',
       'bilirubin_apache',
       ...
       'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob', 'aids',
       'cirrhosis', 'diabetes_mellitus', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis'],
      dtype='object', length=170)

In [361]:
svc_poly = svm.SVC(kernel='poly')
svc_rbf = svm.SVC(kernel='rbf')
svc_sig = svm.SVC(kernel='sigmoid')
svc_lin = svm.LinearSVC()

In [362]:
svc_lin.fit(miss_x_train,miss_y_train)
svc_lin_pred = svc_lin.predict(miss_x_test)
sc_svc_lin = svc_lin.score(miss_x_test,miss_y_test)

In [365]:
svc_poly.fit(miss_x_train,miss_y_train)
svc_poly_pred = svc_poly.predict(miss_x_test)
svc_poly_lin = svc_poly.score(miss_x_test,miss_y_test)

In [366]:
svc_rbf.fit(miss_x_train,miss_y_train)
svc_rbf_pred = svc_rbf.predict(miss_x_test)
sc_svc_rbf = svc_rbf.score(miss_x_test,miss_y_test)

In [367]:
svc_sig.fit(miss_x_train,miss_y_train)
svc_sig_pred = svc_sig.predict(miss_x_test)
sc_svc_sig = svc_sig.score(miss_x_test,miss_y_test)

In [334]:
print("Linear kernel:", sc_svc_lin)
print("Polynomial kernel:", sc_svc_pol)
print("Radio Base Frequency kernel:", sc_svc_rbf)
print("Sigmoid kernel:", sc_svc_sig)


Linear kernel: 0.9260739986915753
Polynomial kernel: 0.9222616658949426
Radio Base Frequency kernel: 0.9287271934287998
Sigmoid kernel: 0.880315475757796


In [335]:
miss_x_train_best = miss_x_train.filter(best_feats)
miss_x_test_best = miss_x_test.filter(best_feats)

In [336]:
svc_poly2 = svm.SVC(kernel='poly')
svc_rbf2 = svm.SVC(kernel='rbf')
svc_sig2 = svm.SVC(kernel='sigmoid')
svc_lin = svm.LinearSVC()

In [337]:
svc_lin2.fit(miss_x_train_best,miss_y_train)
svc_lin_pred2 = svc_lin2.predict(miss_x_test_best)
sc_svc_lin2 = svc_lin2.score(miss_x_test_best,miss_y_test)

In [338]:
svc_poly2.fit(miss_x_train_best,miss_y_train)
svc_poly_pred2 = svc_poly2.predict(miss_x_test_best)
svc_poly_lin2 = svc_poly2.score(miss_x_test_best,miss_y_test)

In [339]:
svc_rbf2.fit(miss_x_train_best,miss_y_train)
svc_rbf_pred2 = svc_rbf2.predict(miss_x_test_best)
sc_svc_rbf2 = svc_rbf2.score(miss_x_test_best,miss_y_test)

In [340]:
svc_sig2.fit(miss_x_train_best,miss_y_train)
svc_sig_pred2 = svc_sig2.predict(miss_x_test_best)
sc_svc_sig2 = svc_sig2.score(miss_x_test_best,miss_y_test)

In [341]:
print("Linear kernel:", sc_svc_lin2)
print("Polynomial kernel:", sc_svc_pol2)
print("Radio Base Frequency kernel:", sc_svc_rbf2)
print("Sigmoid kernel:", sc_svc_sig2)


Linear kernel: 0.9203678127498728
Polynomial kernel: 0.914801666923908
Radio Base Frequency kernel: 0.9210220251508323
Sigmoid kernel: 0.8502943955804317


**RandomizedCV Testing**

In [346]:
dists = {
    "C": uniform(loc=0, scale=3),
    "gamma": uniform(loc=0,scale=(3/171)) # number of features
}

lin_cv = RandomizedSearchCV(
    svm.LinearSVC(random_state=12), 
    {"C": uniform(loc=0, scale=3),"penalty":['l1','l2']},
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=214,
    verbose=0)

poly_cv = RandomizedSearchCV(
    svm.SVC(random_state=12,kernel='poly'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=215,
    verbose=0)

rbf_cv = RandomizedSearchCV(
    svm.SVC(random_state=13,kernel='rbf'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=216,
    verbose=0)

sig_cv = RandomizedSearchCV(
    svm.SVC(random_state=15,kernel='sigmoid'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=217,
    verbose=0)

In [358]:
lin_cv.fit(miss_x_train,miss_y_train)
lin_cv_pred = lin_cv.predict(miss_x_test)
lin_cv_sc = lin_cv.score(miss_x_test,miss_y_test)
lin_cv_results = pd.DataFrame(lin_cv.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increas

In [359]:
poly_cv.fit(miss_x_train,miss_y_train)
poly_cv_pred = poly_cv.predict(miss_x_test)
poly_cv_sc = poly_cv.score(miss_x_test,miss_y_test)
poly_cv_results = pd.DataFrame(poly_cv.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Librar

In [347]:
rbf_cv.fit(miss_x_train,miss_y_train)
rbf_cv_pred = rbf_cv.predict(miss_x_test)
rbf_cv_sc = rbf_cv.score(miss_x_test,miss_y_test)
rbf_cv_results = pd.DataFrame(rbf_cv.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Librar

In [427]:
rbf_cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,...,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall
0,650.712089,6.969153,36.635373,0.210464,0.106767,0.013678,"{'C': 0.10676710492889818, 'gamma': 0.01367797...",0.913629,0.913629,0.913629,...,0.000000,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10
1,391.299244,5.128305,23.364456,0.207180,2.056923,0.003705,"{'C': 2.0569231285180263, 'gamma': 0.003704949...",0.927181,0.927960,0.926636,...,0.017703,5,0.239856,0.229937,0.218215,0.248649,0.225428,0.232417,0.010728,1
2,3913.884127,458.608359,591.594798,448.587815,0.724986,0.01657,"{'C': 0.7249856571992477, 'gamma': 0.016569637...",0.917757,0.916277,0.916277,...,0.036353,3,0.056808,0.042381,0.042381,0.052252,0.038774,0.046519,0.006826,9
3,4456.220874,7.645968,98.958671,1.658096,1.244441,0.013076,"{'C': 1.2444409071927485, 'gamma': 0.013076094...",0.922118,0.922040,0.922975,...,0.020032,7,0.160505,0.153291,0.152390,0.149550,0.153291,0.153805,0.003620,5
4,501.619941,27.277372,84.988784,0.873781,0.402729,0.00743,"{'C': 0.4027290119076675, 'gamma': 0.007430481...",0.921651,0.921028,0.921573,...,0.011045,1,0.111812,0.104599,0.114518,0.107207,0.100090,0.107645,0.005121,7
5,5903.903042,451.975264,986.823764,37.153282,1.340712,0.015277,"{'C': 1.34071229863613, 'gamma': 0.01527705965...",0.920327,0.920327,0.921028,...,0.018134,8,0.135257,0.124436,0.127142,0.120721,0.125338,0.126579,0.004818,6
6,963.555064,14.730845,81.415162,10.265329,1.53282,0.005997,"{'C': 1.5328202292244688, 'gamma': 0.005997140...",0.927025,0.927648,0.926168,...,0.018979,6,0.245266,0.229035,0.221821,0.237838,0.222723,0.231337,0.009015,2
7,8733.772651,26.155030,454.169053,449.505748,0.889659,0.014046,"{'C': 0.8896585620117006, 'gamma': 0.014045534...",0.920405,0.919315,0.919470,...,0.025592,4,0.110911,0.091073,0.091975,0.090991,0.083859,0.093762,0.009058,8
8,1807.235928,478.174613,39.792777,5.191415,2.010891,0.001823,"{'C': 2.010891115338811, 'gamma': 0.0018231769...",0.926090,0.926246,0.925156,...,0.014884,2,0.201082,0.193868,0.186655,0.198198,0.179441,0.191849,0.007881,3
9,8869.047802,14.876884,31.556014,0.562615,1.440763,0.013151,"{'C': 1.4407625884521686, 'gamma': 0.013151306...",0.922196,0.922664,0.923287,...,0.021667,9,0.174031,0.168620,0.166817,0.154955,0.166817,0.166248,0.006235,4


In [348]:
sig_cv.fit(miss_x_train,miss_y_train)
sig_cv_pred = sig_cv.predict(miss_x_test)
sig_cv_sc = sig_cv.score(miss_x_test,miss_y_test)
sig_cv_results = pd.DataFrame(sig_cv.cv_results_)

In [349]:
print("Linear CV kernel:", lin_cv_sc)
print("Polynomial CV kernel:", poly_cv_sc)
print("Radio Base Frequency CV kernel:", rbf_cv_sc)
print("Sigmoid CV kernel:", sig_cv_sc)

Linear CV kernel: 0.9269826270262412
Polynomial CV kernel: 0.9263284146252817
Radio Base Frequency CV kernel: 0.9301809987642654
Sigmoid CV kernel: 0.9049574761939376


In [350]:
dists = {
    "C": uniform(loc=0, scale=3),
    "gamma": uniform(loc=0,scale=(3/171)) # number of features
}

lin_cv2 = RandomizedSearchCV(
    svm.LinearSVC(random_state=12), 
    {"C": uniform(loc=0, scale=3),"penalty":['l1','l2']},
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=214,
    verbose=0)

poly_cv2 = RandomizedSearchCV(
    svm.SVC(random_state=12,kernel='poly'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=215,
    verbose=0)

rbf_cv2 = RandomizedSearchCV(
    svm.SVC(random_state=13,kernel='rbf'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=216,
    verbose=0)

sig_cv2 = RandomizedSearchCV(
    svm.SVC(random_state=15,kernel='sigmoid'), 
    dists,
    n_iter = 10,
    n_jobs=-1,
    scoring=['accuracy','f1','precision','recall'],
    refit='accuracy',
    cv=5,
    random_state=217,
    verbose=0)

In [351]:
lin_cv2.fit(miss_x_train_best,miss_y_train)
lin_cv_pred2 = lin_cv2.predict(miss_x_test_best)
lin_cv_sc2 = lin_cv2.score(miss_x_test_best,miss_y_test)
lin_cv_results2 = pd.DataFrame(lin_cv2.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increas

In [352]:
poly_cv2.fit(miss_x_train_best,miss_y_train)
poly_cv_pred2 = poly_cv2.predict(miss_x_test_best)
poly_cv_sc2 = poly_cv2.score(miss_x_test_best,miss_y_test)
poly_cv_results2 = pd.DataFrame(poly_cv2.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Librar

In [353]:
rbf_cv2.fit(miss_x_train_best,miss_y_train)
rbf_cv_pred2 = rbf_cv2.predict(miss_x_test_best)
rbf_cv_sc2 = rbf_cv2.score(miss_x_test_best,miss_y_test)
rbf_cv_results2 = pd.DataFrame(rbf_cv2.cv_results_)

In [354]:
sig_cv2.fit(miss_x_train_best,miss_y_train)
sig_cv_pred2 = sig_cv2.predict(miss_x_test_best)
sig_cv_sc2 = sig_cv2.score(miss_x_test_best,miss_y_test)
sig_cv_results2 = pd.DataFrame(sig_cv2.cv_results_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Librar

In [355]:
print("Linear CV kernel:", lin_cv_sc2)
print("Polynomial CV kernel:", poly_cv_sc2)
print("Radio Base Frequency CV kernel:", rbf_cv_sc2)
print("Sigmoid CV kernel:", sig_cv_sc2)

Linear CV kernel: 0.9204041578832595
Polynomial CV kernel: 0.9198226357490732
Radio Base Frequency CV kernel: 0.9206585738169659
Sigmoid CV kernel: 0.913898379007051


**True test on the unlabelled data**

In [ ]:
svc_lin_test_pred = svc_lin.predict(testing_X.fillna(0))

In [ ]:
svc_poly_test_pred = svc_poly.predict(testing_X.fillna(0))

In [ ]:
svc_rbf_test_pred = svc_rbf.predict(testing_X.fillna(0))

In [ ]:
svc_sig_test_pred = svc_sig.predict(testing_X.fillna(0))

In [ ]:
# print("Linear kernel:", svc_lin_test_sc)
# print("Polynomial kernel:", svc_poly_test_sc)
# print("Radio Base Frequency kernel:", svc_rbf_test_sc)
# print("Sigmoid kernel:", svc_sig_test_sc)

Linear kernel: 0.998499033275669
Polynomial kernel: 0.5849445405515417
Radio Base Frequency kernel: 2.5440113971710593e-05
Sigmoid kernel: 0.972753637936298
